In [3]:
import pandas as pd
import os

purchaseData = pd.read_csv("Resources/purchase_data.csv")

In [4]:
purchaseData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
Purchase ID    780 non-null int64
SN             780 non-null object
Age            780 non-null int64
Gender         780 non-null object
Item ID        780 non-null int64
Item Name      780 non-null object
Price          780 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


In [5]:
purchaseData.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [6]:
uniquePurchData = purchaseData.nunique(axis=0)

uniquePurchData

Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        183
Item Name      179
Price          145
dtype: int64

In [7]:
pd.DataFrame({
    "Total Players":[purchaseData['SN'].unique().size]
})

#print(type(purchaseData['SN'].unique().size))

,Total Players
0,576


In [8]:
purchaseData.shape[0]

780

In [9]:
formatResults = pd.DataFrame({
    "Number of Unique Items":[purchaseData['Item Name'].unique().size],
    "Average Price":[purchaseData['Price'].mean()],
    "Number of Purchases":[purchaseData.shape[0]],
    "Total Revenue":[purchaseData['Price'].sum()]
})

formatResults['Average Price'] = formatResults['Average Price'].map('${:,.2f}'.format)
formatResults['Total Revenue'] = formatResults['Total Revenue'].map('${:,.2f}'.format)

formatResults

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [10]:
('{:.1f}'.format(purchaseData['Price'].mean()))

'3.1'

In [11]:
import pandas as pd
df = pd.DataFrame([123.4567, 234.5678, 345.6789, 456.7890],
                  index=['foo','bar','baz','quux'],
                  columns=['cost'])
df['foo'] = df['cost']
df['cost'] = df['cost'].map('${:,.2f}'.format)
print(df)

         cost       foo
foo   $123.46  123.4567
bar   $234.57  234.5678
baz   $345.68  345.6789
quux  $456.79  456.7890


In [12]:
formatResults['Average Price'] = formatResults['Average Price'].map('${:,.2f}'.format)

ValueError: Unknown format code 'f' for object of type 'str'

In [ ]:
formatResults

In [ ]:
print(type(formatResults.iloc[0,1]))

print(type(formatResults['Average Price']))

In [ ]:
pd.DataFrame()

In [119]:
gender_demographics = pd.DataFrame({
    'Total Count': purchaseData['Gender'].value_counts(),
    'Percentage of Players': purchaseData['Gender'].value_counts(normalize = True) * 100
})

gender_demographics['Percentage of Players'] = gender_demographics['Percentage of Players'].map('{:.2f}%'.format)

gender_demographics

,Total Count,Percentage of Players
Male,652,83.59%
Female,113,14.49%
Other / Non-Disclosed,15,1.92%


In [13]:
purchaseData.groupby(['Gender'])['Age'].mean()

Gender
Female                   21.345133
Male                     22.917178
Other / Non-Disclosed    24.200000
Name: Age, dtype: float64

In [14]:
gender_analysis = pd.DataFrame({
    'Purchase Count': purchaseData['Gender'].value_counts(),
    'Average Purchase Price': purchaseData.groupby(['Gender'])['Price'].mean(),
    'Total Purchase Value': purchaseData.groupby(['Gender'])['Price'].sum(),
    'Avg Total Purchase per Person': purchaseData.groupby(['Gender', 'SN'])['Price'].sum().groupby(['Gender']).mean()
})

gender_analysis['Average Purchase Price'] = gender_analysis['Average Purchase Price'].map('${:,.2f}'.format)
gender_analysis['Total Purchase Value'] = gender_analysis['Total Purchase Value'].map('${:,.2f}'.format)
gender_analysis['Avg Total Purchase per Person'] = gender_analysis['Avg Total Purchase per Person'].map('${:,.2f}'.format)

gender_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [120]:
test = purchaseData.groupby(['Gender', 'SN'])['Price'].mean().groupby(['Gender']).mean()

test

#test.groupby(['Gender']).mean()

Gender
Female                   3.194835
Male                     3.014269
Other / Non-Disclosed    3.348636
Name: Price, dtype: float64

### Create bins and lables for each bin

In [16]:
bins = [0,10,15,20,25,30,35,40,purchaseData['Age'].max()]
group_names = ['<10',
               '10-14',
               '15-19',
               '20-24',
               '25-29',
               '30-34',
               '35-39',
               '40+'
]

### Bin data by Age and add it as an additional column

In [46]:
binned_age_group = pd.cut(purchaseData['Age'],bins,labels=group_names)

purchaseData['Age Group'] = binned_age_group

purchaseData.head()

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
Age Group      576
dtype: int64

### Remove duplicate users

In [49]:
unique_SN_data = purchaseData.drop_duplicates(subset='SN')

print(unique_SN_data.count())

totalCount = unique_SN_data['SN'].count()

print(totalCount)

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
Age Group      576
dtype: int64
576


### Group data by Age Group and give count

In [48]:
grouped_purchase_data = unique_SN_data.groupby('Age Group')

grouped_purchase_data['SN'].count()

Age Group
<10       24
10-14     41
15-19    150
20-24    232
25-29     59
30-34     37
35-39     26
40+        7
Name: SN, dtype: int64

### Create new data frame containing Age Group count and formatted percentage of total user

In [51]:
age_demographics = pd.DataFrame({
    'Total Count': grouped_purchase_data['SN'].count(),
    'Percentage of Players': grouped_purchase_data['SN'].count() / totalCount * 100
})

age_demographics['Percentage of Players'] = age_demographics['Percentage of Players'].map('{:.2f}%'.format)

age_demographics

,Total Count,Percentage of Players
Age Group,,
<10,24,4.17%
10-14,41,7.12%
15-19,150,26.04%
20-24,232,40.28%
25-29,59,10.24%
30-34,37,6.42%
35-39,26,4.51%
40+,7,1.22%


In [130]:
purchasing_analysis_by_age = pd.DataFrame({
    'Purchase Count': purchaseData.groupby('Age Group')['Price'].count(),
    'Average Purchase Price': purchaseData.groupby('Age Group')['Price'].mean(),
    'Total Purchase Value': purchaseData.groupby('Age Group')['Price'].sum(),
    'Avg Total Purchase per Person': purchaseData.groupby(['Age Group','SN'])['Price'].sum().groupby(['Age Group']).mean() 
})

purchasing_analysis_by_age['Average Purchase Price'] = purchasing_analysis_by_age['Average Purchase Price'].map('${:,.2f}'.format)
purchasing_analysis_by_age['Total Purchase Value'] = purchasing_analysis_by_age['Total Purchase Value'].map('${:,.2f}'.format)
purchasing_analysis_by_age['Avg Total Purchase per Person'] = purchasing_analysis_by_age['Avg Total Purchase per Person'].map('${:,.2f}'.format)

purchasing_analysis_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,32,$3.40,$108.96,$4.54
10-14,54,$2.90,$156.60,$3.82
15-19,200,$3.11,$621.56,$4.14
20-24,325,$3.02,$981.64,$4.23
25-29,77,$2.88,$221.42,$3.75
30-34,52,$2.99,$155.71,$4.21
35-39,33,$3.40,$112.35,$4.32
40+,7,$3.08,$21.53,$3.08


In [60]:
purchaseData.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price',
       'Age Group'],
      dtype='object')

In [128]:
purchaseData.groupby(['SN','Age Group'])['Price'].sum().groupby('Age Group').mean()

Age Group
<10      4.540000
10-14    3.819512
15-19    4.143733
20-24    4.231207
25-29    3.752881
30-34    4.208378
35-39    4.321154
40+      3.075714
Name: Price, dtype: float64

In [114]:
#purchaseData.sort_values(by=['SN']).head(20)
purchaseData.groupby(['Age Group','SN'])['Price'].sum().groupby(['Age Group']).count()

Age Group
<10       24
10-14     41
15-19    150
20-24    232
25-29     59
30-34     37
35-39     26
40+        7
Name: Price, dtype: int64

In [91]:
purchaseData.shape

(780, 8)

In [135]:
purchaseData.groupby(['SN']).count().sort_values(by=['Age'],ascending=False).head()

,Purchase ID,Age,Gender,Item ID,Item Name,Price,Age Group
SN,,,,,,,
Lisosia93,5,5,5,5,5,5,5
Iral74,4,4,4,4,4,4,4
Idastidru52,4,4,4,4,4,4,4
Asur53,3,3,3,3,3,3,3
Inguron55,3,3,3,3,3,3,3


In [145]:
top_spenders = pd.DataFrame({
    'Purchase Count': purchaseData.groupby(['SN'])['Purchase ID'].count(),
    'Average Purchase Price': purchaseData.groupby(['SN'])['Price'].mean(),
    'Total Purchase Value': purchaseData.groupby(['SN'])['Price'].sum()
})

top_spenders = top_spenders.sort_values(by=['Total Purchase Value'],ascending=False).head()

top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${:,.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:,.2f}'.format)

top_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [167]:
most_popular_items = pd.DataFrame({
    'Purchase Count': purchaseData.groupby(['Item ID','Item Name'])['Purchase ID'].count(),
    'Item Price': purchaseData.groupby(['Item ID','Item Name'])['Price'].mean(),
    'Total Purchase Value': purchaseData.groupby(['Item ID','Item Name'])['Purchase ID'].count() * purchaseData.groupby(['Item ID','Item Name'])['Price'].mean()
})

most_popular_items = most_popular_items.sort_values(by=['Purchase Count'],ascending=False)

most_popular_items['Item Price'] = most_popular_items['Item Price'].map('${:,.2f}'.format)
most_popular_items['Total Purchase Value'] = most_popular_items['Total Purchase Value'].map('${:,.2f}'.format)

most_popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [153]:
purchaseData.groupby(['Item ID','Item Name'])['Price']

In [163]:
purchaseData.loc[purchaseData['Price'] > 3].groupby(['Item ID','Item Name'])['Price'].sum()

Item ID  Item Name                                   
1        Crucifer                                         9.78
5        Putrid Fan                                      16.32
6        Rusty Skull                                      7.40
8        Purgatory, Gem of Regret                        11.79
11       Brimstone                                       18.25
                                                         ...  
175      Woeful Adamantite Claymore                      13.44
178      Oathbreaker, Last Hope of the Breaking Storm    50.76
179      Wolf, Promise of the Moonwalker                 26.88
180      Stormcaller                                      3.36
182      Toothpick                                       12.09
Name: Price, Length: 98, dtype: float64